In [1]:
import re, h5py

def infer_positron_ratio(path):
    # try filename like ..._pos0_ or ..._pos1_
    m = re.search(r'_pos(\d)(?:_|\.|$)', path)
    if m:
        return float(m.group(1))
    # fallback: read from HDF5 header if present
    try:
        with h5py.File(path, "r") as f:
            for k in ["/header/positronRatio", "/header/positron_ratio", "/header/positron"]:
                if k in f:
                    return float(f[k][()])
    except Exception:
        pass
    return None


In [2]:
%matplotlib
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os, re, h5py
import numpy as np
from pathlib import Path
#from folderToMovie import *

def _infer_positron_ratio(imageFile: str):
    stem = Path(imageFile).stem  # no .h5
    # case 1: our pattern ..._pos<val>
    m = re.search(r'(?:^|_)pos(?P<val>[\d.]+)$', stem)
    if m:
        return float(m.group('val'))
    # case 2: last token is purely numeric ..._<val>
    last = stem.split('_')[-1]
    try:
        return float(last)
    except ValueError:
        pass
    # case 3: try to read from file (if the build writes it)
    try:
        with h5py.File(imageFile, "r") as H:
            return float(H["header"]["electrons"]["positronRatio"][()])
    except Exception:
        return float("nan")  # unknown but don't crash

def colorbar(mappable):
    """ the way matplotlib colorbar should have been implemented """
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    return fig.colorbar(mappable, cax=cax)

def plotPositronTestFrame(imageFile, intensityMax=3e-3, cpMax=1e-2, output=None, EVPA_CONV="EofN", fractionalCircular=True):

    #Open the IPOLE output and extract relevant data.
    
    with h5py.File(imageFile, 'r') as hfp:
        dx = hfp['header']['camera']['dx'][()]
        dy = hfp['header']['camera']['dy'][()]
        dsource = hfp['header']['dsource'][()]
        lunit = hfp['header']['units']['L_unit'][()]
        fov_muas = dx / dsource * lunit * 2.06265e11
        scale = hfp['header']['scale'][()]
        evpa_0 = 'W'
        if 'evpa_0' in hfp['header']:
            evpa_0 = hfp['header']['evpa_0'][()]
        unpol = np.copy(hfp['unpol']).transpose((1,0)) * scale
        imagep = np.copy(hfp['pol']).transpose((1,0,2)) * scale
        pixelSize = dx * dy * (lunit / dsource * 2.06265e11)**2 / (imagep.shape[0] * imagep.shape[1])
        I = imagep[:,:,0] / pixelSize
        Q = imagep[:,:,1] / pixelSize
        U = imagep[:,:,2] / pixelSize
        V = imagep[:,:,3] / pixelSize
        fov_muas = 1*fov_muas # Find 86GHz FoV, 230GHz FoV in ipole_positron_Test_unlocked.py; Increasing fov_muas here zooms in                                         
    extent = [ -1*fov_muas/2, 1*fov_muas/2, -1*fov_muas/2, 1*fov_muas/2 ]

    #Initialize a plot with two panels.
    fig, axarr = plt.subplots(1, 2, figsize=(8,4))
    ax1 = axarr[0]
    ax2 = axarr[1]

    #Total intensity and linear polarization ticks.
    #Is = np.sum(I)
    im1 = ax1.imshow(I, cmap='afmhot', vmin=0., vmax=np.max(I), origin='lower', extent=extent) #vmax=intensityMax
    # intensityMax; vmax=np.max(I)
    colorbar(im1)
    #print(Is)
    #print('V/I = {0:1.2e}'.format(np.sum(V)/np.sum(I)))
    
    #Circular polarization fraction
    if fractionalCircular:
        cpfrac = 100.*V/I
        im2 = ax2.imshow(cpfrac, cmap='seismic', vmin=-cpMax, vmax=cpMax, origin='lower', extent=extent)
        colorbar(im2)
        ax2.set_title("CP [%]")
    else:
        im2 = ax2.imshow(V, cmap='seismic', vmin=-cpMax/100.0*intensityMax, vmax=cpMax/100.0*intensityMax, origin='lower', extent=extent)
        colorbar(im2)
        ax2.set_title("CP [Jy $\mu$as$^{-2}$]")

    #evpa
    evpa = (180./3.14159)*0.5*np.arctan2(U,Q)
    if evpa_0 == "W":
        evpa += 90.
        evpa[evpa > 90.] -= 180.
    if EVPA_CONV == "NofW":
        evpa += 90.
        evpa[evpa > 90.] -= 180.

    #quiver on intensity
    npix = I.shape[0]
    xs = np.linspace(-fov_muas/2,fov_muas/2,npix)
    Xs,Ys = np.meshgrid(xs,xs)
    lpscal = np.max(np.sqrt(Q*Q+U*U))
    vxp = np.sqrt(Q*Q+U*U)*np.sin(evpa*3.14159/180.)/lpscal
    vyp = -np.sqrt(Q*Q+U*U)*np.cos(evpa*3.14159/180.)/lpscal
    skip = int(npix/32)
    ax1.quiver(Xs[::skip,::skip],Ys[::skip,::skip],vxp[::skip,::skip],vyp[::skip,::skip],
      headwidth=1, headlength=1,
      width=0.005,
      color='#00ff00',
      units='width',
      scale=4,
      pivot='mid')

    #Formatting
    ax1.set_title("Stokes I [Jy $\mu$as$^{-2}$]")
    ax1.set_ylabel('$\mu$as')
    ax1.set_xlabel('$\mu$as')
    ax2.set_xlabel('$\mu$as')
    for axis in [ax1,ax2]:
        axis.set_aspect('equal')
        axis.set_xlim([-1*40,1*40])
        axis.set_ylim([-1*40,1*40])
        axis.set_xticks(np.linspace(-1*40,1*40,1*5))
        axis.set_yticks(np.linspace(-1*40,1*40,1*5))

    #Label.  Note that the positron fraction was included as part of the file name.  See some basic calculations here.
    positronRatio = _infer_positron_ratio(imageFile)
    bbox = {'boxstyle': 'round', 'facecolor': 'wheat', 'alpha': 0.8}
    ax2.text(0.05, 0.05, r'$n_{pairs}/(n_-)_0=$' + '{0:1.2f}'.format(positronRatio), ha='left', va='bottom', transform=ax2.transAxes, fontsize=12)
    ax2.text(0.05, 0.95, 'V/I = {0:1.2e}'.format(np.sum(V)/np.sum(I)), ha='left', va='top', transform=ax2.transAxes, fontsize=12)
    ax1.text(0.05, 0.95, 'I={0:3.2e} Jy'.format(np.sum(I) * pixelSize), ha='left', va='top', transform=ax1.transAxes, fontsize=12, color='white')
    #ax1.text(0.05, 0.95, 'I={0:3.2f} Jy'.format(np.sum(I) * pixelSize), ha='left', va='top', transform=ax1.transAxes, fontsize=12, color='white')
    ax1.text(0.05, 0.05, 'P/I={0:1.2e}'.format(np.sqrt(np.sum(Q)**2 + np.sum(U)**2)/np.sum(I)), ha='left', va='bottom', transform=ax1.transAxes, fontsize=12, color='white')
    print(positronRatio, '{0:1.2e}'.format(np.sum(V)/np.sum(I)))
    fig.tight_layout()
    fig.savefig(imageFile.replace(".h5",".png"))
    '''if output is None:
        fig.show()
    else:
        fig.savefig(imageFile.replace(".h5",".png"))
        plt.close(fig)'''


Using matplotlib backend: agg


In [3]:
imageFile = '/work/vmo703/ipole_outputs/output_Sa-0.5_5000_model_RBETA_Rhigh_40_0.000.h5'
imageFile2 = '/work/vmo703/ipole_outputs/output_Sa-0.5_5000_model_RBETA_Rhigh_40_1.000.h5'

plotPositronTestFrame(imageFile, cpMax=0.1, fractionalCircular=False)
plotPositronTestFrame(imageFile2, cpMax=0.1, fractionalCircular=False)

0.0 -5.27e-03
1.0 -5.55e-03


In [ ]:
import glob

def generate_all_images(directory, cpMax=0.1):
    # look through all /ipole_outputs/*/ subdirs
    dump_dirs = [os.path.join(directory, d) for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]

    for dump_dir in dump_dirs:
        images_dir = os.path.join(dump_dir, "images")
        os.makedirs(images_dir, exist_ok=True)

        # find all *_0.h5 and *_1.h5 files in this subdir
        for r in [0,1]:
            h5_files = glob.glob(os.path.join(dump_dir, f"*_{r}.h5")) # e.g. *_0.h5, *_1.h5
            for h5_file in h5_files:
                try:
                    print(f"plotting {h5_file}")
                    plotPositronTestFrame(
                        h5_file,
                        cpMax=cpMax,
                        fractionalCircular=False,
                        save=True,
                        outdir=images_dir
                    )
                except Exception as e:
                    print(f"failed to plot {h5_file}: {e}")
                    